In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

# Iterators for linked lists


Recall that an iterator over a list sits between elements in the list. The figure below shows a list having 6 elements and an iterator in red between the two
elements `"c"` and `"d"`:

![](../resources/images/linked_list_iterator/iterator_1.png)

Recall that an iterator somewhere in the middle of the list has *previous* and *next* elements. In the `ArrayIterator` implementation, there
were actually fields called `prev` and `next` that were used to represent the state of the iterator.

In a linked list, it is not necessary to maintain a reference to the next element in the iteration if we maintain a reference to the node
immediately before the iterator because we can obtain the next element by following the link from the previous node. In our implementation,
the field `prev` will always be a reference to the node immediately before the iterator; its main responsibility is to keep track
of the position of the iterator.

Recall that the `remove` method removes the element that was most recently returned by `next` and can be called only once for each call to `next`.
In our implementation, `remove` removes the node `prev`. To efficiently remove `prev`, it is necessary to maintain a reference to the node
immediately before `prev`. We use the field `prevPrev` (short for *previous* to *`prev`*) to maintain a reference to the node immediately before `prev`.
The figure below shows the nodes referenced by `pref` and `prevPrev`.

![](../resources/images/linked_list_iterator/iterator_2.png)

There is no node immediately before the iterator at the start of an iteration because there is no node before the head node of the list.
To avoid having to deal with special cases where `prev == null` it would be nice if we could enforce the invariant that `prev != null` is
always `true`. To do so, we can have the iterator class create a node in front of the head node whose only purpose is to ensure that
`prev` is never `null` as shown in the figure below:

![](../resources/images/linked_list_iterator/iterator_3.png)

Notice that `prevPrev` can be `null` and we intentionally set `prevPrev = null` at the start of an iteration.

Implementing `hasNext` is easy: `hasNext` returns `true` when `prev.next` is not `null` (i.e., when there is another node
after `prev`). Because `prev` is never `null`, we can always write `prev.next` without a `NullPointerException` being thrown.

Implementing `next` is also easy. If `hasNext` returns `false`, then throw a `NoSuchElementException`. Otherwise, the element
to be returned can be obtained as `prev.next.elem`. Advancing the iterator is performed by advancing `prevPrev` and `prev`
one node to the right.

Implementing `remove` is slightly tricky. Consider the case where the iterator is in the middle of the list as shown in the figure below:

![](../resources/images/linked_list_iterator/iterator_4.png)

`remove` should remove the node referred to by `prev`. To perform the removal, we call the `SLinkedList` method `removeAfter(this.prevPrev)`
which removes the node as shown in the figure below:

![](../resources/images/linked_list_iterator/iterator_5.png)

In our implementation, `prev` always refers to the node immediately before the iterator; thus, we must assign `prev = prevPrev`. Finally,
to indicate that `remove` has been called once for the most recent call to `next` we set `prevPrev = null` as shown in the figure below:

![](../resources/images/linked_list_iterator/iterator_6.png)

Now we have a way to test if `remove` cannot be safely called: Simply test if `prevPrev == null` is `true` and if so, throw an
`IllegalStateException`.

There is one more special case that we have to handle in the `remove` method. Recall that in the linked list version of `remove`,
removing the first element of the list is a special case because the `head` field must be updated. This situation also occurs
for the iterator version of `remove`.

The figure below illustrates an iterator where the `next` method has been called exactly once.

![](../resources/images/linked_list_iterator/iterator_7.png)

Calling the iterator `remove` method should remove the head node of the list which is a special case.
To handle the special case we call the `SLinkedList` method `removeFront` which adjusts the head node and size of the list;
however, it does not actually delete the former head node from memory (because there is no way to manually delete an object
in Java). `prev` still refers to the former head node of the list as shown in the figure below:

![](../resources/images/linked_list_iterator/iterator_8.png)

Because `remove` has been called, we set `prevPrev = null`. `prev` correctly refers to the node immediately before the iterator,
but the element stored in the node is no longer required; therefore, we set `this.prev.elem = null` to allow the virtual machine
to garbage collect the memory associated with the removed element:

![](../resources/images/linked_list_iterator/iterator_9.png)

## A complete linked list implementation

The complete `SLinkedList` and its inner `LLIterator` class is shown below:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Iterator;
import java.util.NoSuchElementException;
import java.util.StringJoiner;

import ca.queensu.cs.cisc124.notes.generics.list.SList;

public class SLinkedList<E> implements SList<E> {

    static class Node<E> {
        E elem;
        Node<E> next;

        /**
         * Initializes a node to refer to the specified element and node.
         * 
         * @param c a character
         */
        public Node(E elem, Node<E> node) {
            this.elem = elem;
            this.next = node;
        }
    }

    /**
     * The number of elements in the linked list.
     */
    private int size;

    /**
     * The first node of the linked list; will be <code>null</code> for an empty
     * list.
     */
    private Node<E> head;

    /**
     * The last node of the linked list; will be <code>null</code> for an empty
     * list.
     */
    private Node<E> tail;

    
    /**
     * Returns the head node of this list.
     * 
     * @return the head node of this list
     */
    Node<E> head() {
        return this.head;
    }
    
    /**
     * Returns the tail node of this list.
     * 
     * @return the tail node of this list
     */
    Node<E> tail() {
        return this.tail;
    }
    
    
    /**
     * Initialize an empty list.
     */
    public SLinkedList() {
        this.size = 0;
        this.head = null;
        this.tail = null;
    }

    /**
     * Get the number of elements in the list.
     * 
     * @return the number of elements in the list.
     */
    @Override
    public int size() {
        return this.size;
    }
    
    /**
     * Adds the given element to the end of the list.
     * 
     * @param elem the element to add
     */
    @Override
    public void add(E elem) {
        if (this.size == 0) {
            this.head = new Node<>(elem, null);
            this.tail = this.head;
        } else {
            Node<E> n = new Node<>(elem, null);
            this.tail.next = n;
            this.tail = n;
        }
        this.size++;
    }

    /**
     * Validates the specified index.
     * 
     * @param index an index
     * @throws IndexOutOfBoundsException if
     *                                   {@code index < 0 || index >= this.size()}
     */
    void validate(int index) {
        if (index < 0 || index >= this.size) {
            throw new IndexOutOfBoundsException("index out of bounds: " + index);
        }
    }

    /**
     * Returns the node at the specified index. Assumes that the index is valid for
     * this list to avoid re-validating the index.
     * 
     * @param index a valid index for this list
     * @return the node at the specified index
     */
    Node<E> moveTo(int index) {
        Node<E> n = this.head;
        for (int i = 0; i < index; i++) {
            n = n.next;
        }
        return n;
    }

    /**
     * Returns the element at the specified position in the list.
     * 
     * @param index index of the element to return
     * @return the element at the specified position
     * @throws IndexOutOfBoundsException if the index is out of the range
     *                                   {@code (index < 0 || index >= size())}
     */
    @Override
    public E get(int index) {
        this.validate(index);
        Node<E> n = this.moveTo(index);
        return n.elem;
    }

    /**
     * Sets the element at the specified position in the list.
     * 
     * @param index index of the element to set
     * @param elem element to be stored at the specified position
     * @throws IndexOutOfBoundsException if the index is out of the range
     *                                   {@code (index < 0 || index >= size())}
     */
    @Override
    public E set(int index, E elem) {
        this.validate(index);
        Node<E> n = this.moveTo(index);
        E old = n.elem;
        n.elem = elem;
        return old;
    }

    /**
     * Adds an element to the front of this list.
     * 
     * @param elem the element to add
     */
    public void addFront(E elem) {
        Node<E> toAdd = new Node<>(elem, null);
        toAdd.next = this.head;
        this.head = toAdd;
        this.size++;
    }

    /**
     * Inserts an element at the specified index of this list. Shifts the element
     * currently at that position (if any) and any subsequent elements to the right.
     * 
     * @param index the index at which to add the element
     * @param elem  the element to add
     * @throws IndexOutOfBoundsException if the index is out of the range
     *                                   {@code (index < 0 || index > size())}
     */
    @Override
    public void add(int index, E elem) {
        
        // WARNING: THERE IS A BUG IN THIS METHOD
        
        if (index == this.size) {
            // add to end
            this.add(elem);
            //return;             bug here
        }
        // must validate after the previous if statement otherwise
        // an exception will be thrown because this.size is not a valid index
        this.validate(index);

        if (index == 0) {
            this.addFront(elem);
        } else {
            Node<E> n = this.moveTo(index - 1);
            Node<E> toAdd = new Node<>(elem, null);
            Node<E> after = n.next;
            n.next = toAdd;
            toAdd.next = after;
            this.size++;
        }
    }

    /**
     * Removes the first element of this list and returns the element.
     * 
     * @return the removed element
     * @throws NoSuchElementException if the list is empty
     */
    public E removeFront() {
        if (this.size == 0) {
            throw new NoSuchElementException("list is empty");
        }
        Node<E> toRemove = this.head;
        this.head = toRemove.next;
        // special case of removing from a list of size 1
        if (this.size == 0) {
            this.tail = null;
        }
        this.size--;
        return toRemove.elem;
    }

    /**
     * Removes the element at the specified index of this list, shifts any
     * subsequent elements to the left (subtracts one to their indices), and returns
     * a reference to the removed element.
     * 
     * @param index the index of the element to remove
     * @return the removed element
     * @throws IndexOutOfBoundsException if the index is out of the range
     *                                   {@code (index < 0 || index >= size())}
     */
    @Override
    public E remove(int index) {
        if (index == 0) {
            return this.removeFront();
        }
        this.validate(index);
        Node<E> n = this.moveTo(index - 1);
        return this.removeAfter(n);
        
    }
    
    /**
     * Removes the node immediately after the specified node.
     * 
     * @param n the node in front of the node to be removed
     * @return the element in the removed node
     */
    E removeAfter(Node<E> n) {
        Node<E> toRemove = n.next;
        Node<E> after = toRemove.next;
        toRemove.next = null;
        n.next = after;
        // special case where the last element was removed
        if (after == null) {
            this.tail = n;
        }
        this.size--;
        return toRemove.elem;
    }
    
    @Override
    public String toString() {
        
        StringJoiner j = new StringJoiner(", ", "[", "]");
        Node<E> n = this.head;
        for (int i = 0; i < this.size; i++) {
            j.add(n.elem.toString());
            n = n.next;
        }
        return j.toString();
    }
    
    /**
     * Returns an iterator over the elements in this list. The iterator visits
     * the elements of this list in the order that the elements appear in this list.
     * 
     * @return an iterator over the elements in this list
     */
    @Override
    public Iterator<E> iterator() {
        return new LLIterator();
    }
    
    
    private class LLIterator implements Iterator<E> {
        /**
         * Node holding element immediately before the iterator
         */
        private Node<E> prev;
        
        /**
         * Node immediately before prev
         */
        private Node<E> prevPrev;
        
        LLIterator() {
            this.prev = new Node<>(null, SLinkedList.this.head);
            this.prevPrev = null;
        }
        
        @Override
        public boolean hasNext() {
            return this.prev.next != null;
        }
        
        @Override
        public E next() {
            if (!this.hasNext()) {
                throw new NoSuchElementException();
            }
            E e = this.prev.next.elem;
            this.prevPrev = this.prev;
            this.prev = this.prev.next;
            return e;
        }
        
        @Override
        public void remove() {
            if (this.prevPrev == null) {
                throw new IllegalStateException();
            }
            if (this.prev == SLinkedList.this.head) {
                SLinkedList.this.removeFront();
                this.prev.elem = null;
            }
            else {
                SLinkedList.this.removeAfter(this.prevPrev);
                this.prev = this.prevPrev;
            }
            this.prevPrev = null;
        }
    }

}


A small test program that uses an iterator to filter out the negative values from a list is shown below:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Iterator;

import ca.queensu.cs.cisc124.notes.generics.list.SList;

SList<Integer> t = new SLinkedList<>();
for (int i = -5; i <= 6; i++) {
    t.add(i);
}
System.out.println(t);

for (Iterator<Integer> iter = t.iterator(); iter.hasNext(); ) {
    Integer val = iter.next();
    if (val < 0) {
        iter.remove();    // can call remove once for each call to next
    }
}
System.out.println(t);

**Exercise 1** In the `SLinkedList` class, removing the tail node required special handling because the `tail` field must be updated. How does the
implementation of the iterator `remove` method avoid explicitly dealing with removal of the tail node?

**Exercise 2** The iterator implementation described above uses a special node in front of the head node of the list so that `prev` is never equal to `null`.
It is possible to implement the iterator without using the special node and allowing `prev` to equal `null`. How does the implementation of `hasNext`
change in this case?

**Exercise 3** The iterator described above uses two references (`prev` and `prevPrev`) to iterate over a linked list. This turns out to be a common
strategy for solving certain types of linked list problems. For example, suppose that you have a linked list where you do not know the size of the list:
How can you get the second last element of the list using only one iteration of the nodes of the list? The third last element?

**Exercise 4** Another problem that can be solved using two references to iterate over a linked list: Suppose that you have a linked list where 
you do not know the size of the list: Provide an algorithm that determines if the linked list has a cycle. A linked list has a cycle if the sequence
loops back on itself (every node has a non-null next link).